# 0. Data load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Dacon_RF_parameter_tuning

/content/drive/MyDrive/Dacon_RF_parameter_tuning


In [ ]:
ls

 0325_EDA_Gridsearch.ipynb   grid_test2.ipynb        submit0327_2.csv          train.csv
 0326_Generalization.ipynb   sample_submission.csv   submit.ipynb
 0327_RandomSearch.ipynb     submit/                 test_grid.ipynb
 EDA.ipynb                   submit0327_1.csv       'test_grid.ipynb의 사본'


In [ ]:
# 대회 제공 시드로 고정
import random
random.seed(42)

In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame

In [ ]:
train = pd.read_csv('train.csv')
train.head(10)

,person_id,Sex,past_login_total,past_1_month_login,past_1_week_login,sub_size,email_type,phone_rat,apple_rat,login
0,0,0,3.0,1.0,1.0,0.0,0,0.000000,0.000000,1
1,1,1,111.0,26.0,7.0,2.0,0,0.072072,0.000000,1
2,3,1,13.0,13.0,11.0,7.0,0,0.076923,1.000000,1
3,4,1,28.0,12.0,5.0,0.0,0,0.071429,0.071429,1
4,5,1,4.0,4.0,4.0,0.0,2,0.000000,0.000000,1
5,6,1,4.0,3.0,2.0,0.0,0,0.000000,0.000000,1
6,7,1,2.0,2.0,1.0,0.0,0,0.000000,1.000000,1
7,8,1,0.0,0.0,0.0,0.0,0,0.000000,0.000000,1
8,9,1,7.0,6.0,6.0,0.0,1,0.000000,0.571429,1
9,10,1,7.0,2.0,2.0,0.0,0,0.000000,0.000000,1


# 1. 편향 데이터 과적합 방지 - Generalization
- EDA 단계에서, 데이터가 심하게 편향되어 있는 것을 확인했다.
- 하이퍼파라미터에 여러 값을 대입하며 성능을 출력하니, 1.0으로 나타났다. -> 과적합 의심
- 과적합 방지와, 편향 데이터를 처리한다는 점에 중점을 두고 하이퍼파라미터를 설정하는 과정을 진행해 보았다.

1-1) 하이퍼파라미터 정리
- n_estimators
 - 결정트리 개수
 - 대체로 커질수록 성능 향상, 무조건적이지는 않음
- criterion = 'gini'
 - 평가 지표
 - 보통은 둘 다 비슷, 특정 데이터에서는 차이나기도 함
- max_depth = 5
 - 트리의 깊이
 - 작으면 과적합 방지
- min_samples_split = 30
 - 노드 분할을 위한 최소한의 데이터 수
 - 작을수록 과적합 가능성 높음
- min_samples_leaf = 30
 - 리프노드 되기 위한 최소한의 샘플 데이터
 - 데이터 양이 불균형하면 작게 설정 -> 크면 과적합 방지
- min_weight_fraction_leaf = 0
 -  min_sample_leaf와 같지만 가중치가 부여된 샘플 수에서의 비율
 - 편향된 데이터에서는 작아야됨
- max_features = 3
 - 데이터의 feature를 참조할 비율
 - 편향된 데이터에서는 작아야됨
- max_leaf_nodes = 2
 - 리프 노드의 최대수
 - 편향된 데이터에서는 작아야됨
- min_impurity_decrease = 0
 - 최소 불순도
 - 편향된 데이터에서는 작아야됨
- bootstrap = False
 - 부트스트랩 여부
 - 편향된 데이터에서는 False

1-2) code

In [ ]:
# data split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 종속변수(target)의 컬럼을 target으로의 선언이 필요합니다.
target = train['login']
X_train, X_test, y_train, y_test = train_test_split(train, target,
                                                      test_size=0.2,
                                                      shuffle=True,
                                                      stratify=target)
#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
rf_clf1 = RandomForestClassifier(n_estimators = 100, # 크면 성능 좋고 오래걸림?
                                 criterion = 'gini',
                                max_depth = 12,
                                min_samples_split = 8, # 작을수록 과적합 가능성 높음
                                 min_samples_leaf = 8, # 데이터 양이 불균형하면 작게 설정
                                 min_weight_fraction_leaf = 0,
                                 max_features = None, # 깊이 깊어지면 과적합 가능성 있음
                                max_leaf_nodes = 2,
                                 min_impurity_decrease = 0,
                                bootstrap = True)


rf_clf1.fit(X_train, y_train)
pred = rf_clf1.predict(X_test)
train_pred = rf_clf1.predict(X_train)
print('train 예측 정확도: {:.4f}'.format(accuracy_score(y_train,train_pred)))
print('예측 정확도: {:.4f}'.format(accuracy_score(y_test,pred)))

train 예측 정확도: 1.0000
예측 정확도: 1.0000


In [ ]:
# data split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 종속변수(target)의 컬럼을 target으로의 선언이 필요합니다.
target = train['login']
X_train, X_test, y_train, y_test = train_test_split(train, target,
                                                      test_size=0.2,
                                                      shuffle=True,
                                                      stratify=target)

#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
rf_clf1 = RandomForestClassifier(n_estimators = 100, # 크면 성능 좋고 오래걸림?
                                 criterion = 'gini',
                                max_depth = 12,
                                min_samples_split = 100, # 작을수록 과적합 가능성 높음
                                 min_samples_leaf = 8, # 데이터 양이 불균형하면 작게 설정
                                 min_weight_fraction_leaf = 0,
                                 max_features = 2, # 깊이 깊어지면 과적합 가능성 있음
                                max_leaf_nodes = 2,
                                 min_impurity_decrease = 0,
                                bootstrap = True)


rf_clf1.fit(X_train, y_train)
pred = rf_clf1.predict(X_test)
train_pred = rf_clf1.predict(X_train)
print('train 예측 정확도: {:.4f}'.format(accuracy_score(y_train,train_pred)))
print('예측 정확도: {:.4f}'.format(accuracy_score(y_test,pred)))

train 예측 정확도: 0.9140
예측 정확도: 0.9160


In [ ]:
# data split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 종속변수(target)의 컬럼을 target으로의 선언이 필요합니다.
target = train['login']
X_train, X_test, y_train, y_test = train_test_split(train, target,
                                                      test_size=0.2,
                                                      shuffle=True,
                                                      stratify=target)

#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
rf_clf1 = RandomForestClassifier(n_estimators = 500, # 크면 성능 좋고 오래걸림?
                                 criterion = 'gini',
                                max_depth = 12,
                                min_samples_split = 100, # 작을수록 과적합 가능성 높음
                                 min_samples_leaf = 8, # 데이터 양이 불균형하면 작게 설정
                                 min_weight_fraction_leaf = 0,
                                 max_features = 2, # 깊이 깊어지면 과적합 가능성 있음
                                max_leaf_nodes = 2,
                                 min_impurity_decrease = 0,
                                bootstrap = True)


rf_clf1.fit(X_train, y_train)
test_pred = rf_clf1.predict(X_test)
train_pred = rf_clf1.predict(X_train)
print('train 예측 정확도: {:.4f}'.format(accuracy_score(y_train,train_pred)))
print('test 예측 정확도: {:.4f}'.format(accuracy_score(y_test,test_pred)))

train 예측 정확도: 0.9255
test 예측 정확도: 0.9160


In [ ]:
# 현재 최고 점수 파라미터 조합 - test4
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 종속변수(target)의 컬럼을 target으로의 선언이 필요합니다.
target = train['login']
X_train, X_test, y_train, y_test = train_test_split(train, target,
                                                      test_size=0.2,
                                                      shuffle=True,
                                                      stratify=target)

#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
rf_clf1 = RandomForestClassifier(n_estimators = 500, # 크면 성능 좋고 오래걸림?
                                 criterion = 'gini',
                                max_depth = None,
                                min_samples_split = 2, # 작을수록 과적합 가능성 높음
                                 min_samples_leaf = 1, # 데이터 양이 불균형하면 작게 설정
                                 min_weight_fraction_leaf = 0,
                                 max_features = 'auto', # 깊이 깊어지면 과적합 가능성 있음
                                max_leaf_nodes = None,
                                 min_impurity_decrease = 0,
                                bootstrap = True)


rf_clf1.fit(X_train, y_train)
test_pred = rf_clf1.predict(X_test)
train_pred = rf_clf1.predict(X_train)
print('train 예측 정확도: {:.4f}'.format(accuracy_score(y_train,train_pred)))
print('test 예측 정확도: {:.4f}'.format(accuracy_score(y_test,test_pred)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


train 예측 정확도: 1.0000
test 예측 정확도: 1.0000


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 종속변수(target)의 컬럼을 target으로의 선언이 필요합니다.
target = train['login']
X_train, X_test, y_train, y_test = train_test_split(train, target,
                                                      test_size=0.2,
                                                      shuffle=True,
                                                      stratify=target)

#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
rf_clf1 = RandomForestClassifier(n_estimators = 500, # 크면 성능 좋고 오래걸림?
                                 criterion = 'gini',
                                max_depth = 5, # 작으면 과적합 방지
                                min_samples_split = 50, # 작을수록 과적합 가능성 높음
                                 min_samples_leaf = 50, # 데이터 양이 불균형하면 작게 설정 -> 크면 과적합 방지
                                 min_weight_fraction_leaf = 0,
                                 max_features = 3, # 깊이 깊어지면 과적합 가능성 있음
                                max_leaf_nodes = None,
                                 min_impurity_decrease = 0,
                                bootstrap = True)


rf_clf1.fit(X_train, y_train)
test_pred = rf_clf1.predict(X_test)
train_pred = rf_clf1.predict(X_train)
print('train 예측 정확도: {:.4f}'.format(accuracy_score(y_train,train_pred)))
print('test 예측 정확도: {:.4f}'.format(accuracy_score(y_test,test_pred)))

train 예측 정확도: 0.9503
test 예측 정확도: 0.9466


In [ ]:
# test 중 점수 파라미터 조합
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 종속변수(target)의 컬럼을 target으로의 선언이 필요합니다.
target = train['login']
X_train, X_test, y_train, y_test = train_test_split(train, target,
                                                      test_size=0.2,
                                                      shuffle=True,
                                                      stratify=target)

#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
rf_clf1 = RandomForestClassifier(n_estimators = 500, # 크면 성능 좋고 오래걸림?
                                 criterion = 'gini', # 보통은 둘 다 비슷, 특정 데이터에서는 차이나기도 함
                                max_depth = 5, # 작으면 과적합 방지
                                min_samples_split = 30, # 작을수록 과적합 가능성 높음
                                 min_samples_leaf = 30, # 데이터 양이 불균형하면 작게 설정 -> 크면 과적합 방지
                                 min_weight_fraction_leaf = 0, # 작아야됨
                                 max_features = 3,  # 편향된 데이터에서는 작아야됨
                                max_leaf_nodes = None, # 작아야됨
                                 min_impurity_decrease = 0, # 작아야됨
                                bootstrap = True) # 편향된 데이터에서는 False


rf_clf1.fit(X_train, y_train)
test_pred = rf_clf1.predict(X_test)
train_pred = rf_clf1.predict(X_train)
print('train 예측 정확도: {:.4f}'.format(accuracy_score(y_train,train_pred)))
print('test 예측 정확도: {:.4f}'.format(accuracy_score(y_test,test_pred)))

train 예측 정확도: 1.0000
test 예측 정확도: 1.0000


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

y_true = [0, 1, 2, 0, 1, 2, 0, 1, 2]
y_pred = [0, 0, 2, 0, 2, 2, 1, 1, 2]
cm = confusion_matrix(y_true, y_pred)
print(cm)

In [ ]:
df = {'n_estimators':500,
      'criterion' : 'gini',
      'max_depth' : 5,
      'min_samples_split' : 30,
      'min_samples_leaf' : 30,
      'min_weight_fraction_leaf' : 0,
      'max_features' : 3,
      'max_leaf_nodes' : 'None',
      'min_impurity_decrease' : 0.0,
      'bootstrap': False
      }
df = DataFrame(df, index=['a'])
df
df.to_csv("./submit0326_5.csv", index=False)

In [ ]:
# test 중 점수 파라미터 조합
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 종속변수(target)의 컬럼을 target으로의 선언이 필요합니다.
target = train['login']
X_train, X_test, y_train, y_test = train_test_split(train, target,
                                                      test_size=0.2,
                                                      shuffle=True,
                                                      stratify=target)

#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
rf_clf1 = RandomForestClassifier(n_estimators = 500, # 크면 성능 좋고 오래걸림?
                                 criterion = 'gini', # 보통은 둘 다 비슷, 특정 데이터에서는 차이나기도 함
                                max_depth = 5, # 작으면 과적합 방지
                                min_samples_split = 30, # 작을수록 과적합 가능성 높음
                                 min_samples_leaf = 30, # 데이터 양이 불균형하면 작게 설정 -> 크면 과적합 방지
                                 min_weight_fraction_leaf = 0, # 작아야됨
                                 max_features = 3,  # 편향된 데이터에서는 작아야됨
                                max_leaf_nodes = 2, # 작아야됨
                                 min_impurity_decrease = 0, # 작아야됨
                                bootstrap = False) # 편향된 데이터에서는 False


rf_clf1.fit(X_train, y_train)
test_pred = rf_clf1.predict(X_test)
train_pred = rf_clf1.predict(X_train)
print('train 예측 정확도: {:.4f}'.format(accuracy_score(y_train,train_pred)))
print('test 예측 정확도: {:.4f}'.format(accuracy_score(y_test,test_pred)))

train 예측 정확도: 0.9322
test 예측 정확도: 0.9198


# 2. 제출

In [ ]:
cd submit

/content/drive/MyDrive/Dacon_RF_parameter_tuning/submit


In [ ]:
ls

submit0326_1.csv  submit0326_3.csv  submit0326_5.csv  test3.csv  test5.csv  test.csv
submit0326_2.csv  submit0326_4.csv  test2.csv         test4.csv  test6.csv


In [ ]:
# 제출 1. n-estimators=600, 나머지도 전부 변경
submit1 = pd.read_csv('submit0326_1.csv')
submit1

,n_estimators,criterion,max_depth,min_samples_split,min_samples_leaf,min_weight_fraction_leaf,max_features,max_leaf_nodes,min_impurity_decrease,bootstrap
0,600,gini,90,12,14,0.1,auto,2,0.01,True


In [ ]:
# 제출 2. submit0326_1에서 n-estimators=300
submit2 = pd.read_csv('submit0326_2.csv')
submit2

,n_estimators,criterion,max_depth,min_samples_split,min_samples_leaf,min_weight_fraction_leaf,max_features,max_leaf_nodes,min_impurity_decrease,bootstrap
0,300,gini,90,12,14,0.1,auto,2,0.01,True


In [ ]:
# 제출 3. grid search -> submit sample에서 min_weight_fraction만 0.0->0.1변경
submit3 = pd.read_csv('submit0326_3.csv')
submit3

,n_estimators,criterion,max_depth,min_samples_split,min_samples_leaf,min_weight_fraction_leaf,max_features,max_leaf_nodes,min_impurity_decrease,bootstrap
0,100,gini,None,2,1,0.1,auto,None,0.0,True


In [ ]:
# 제출 4. - 데이터의 편향 고려한 파라미터 설정 / 현재까지 submit중 최고 점수
submit4 = pd.read_csv('submit0326_4.csv')
submit4

,n_estimators,criterion,max_depth,min_samples_split,min_samples_leaf,min_weight_fraction_leaf,max_features,max_leaf_nodes,min_impurity_decrease,bootstrap
0,500,gini,5,30,30,0,3,None,0.0,True


In [ ]:
# 제출 5. 편향된 데이터에서는 bootstrap을 하지 않아야 한다고 해서 False로 변경
submit5 = pd.read_csv('submit0326_5.csv')
submit5

,n_estimators,criterion,max_depth,min_samples_split,min_samples_leaf,min_weight_fraction_leaf,max_features,max_leaf_nodes,min_impurity_decrease,bootstrap
0,500,gini,5,30,30,0,3,None,0.0,False


# 3. 제출 결과

- 제출 1~3
 - 제출 1 score : 0.7508702532
 - 제출 2 score : 0.7508702532
 - 제출 3 score : 0.7433148734
 - 여전히 grid_search와 직접 대입해본 파라미터 값을 기반으로 submit. 전일 대비 성능 향상 없음

- 제출 4~5
 - 제출 4 score : 0.7852848101
 - 제출 5 score : 0.7852848101
 - 데이터의 편향된 특성을 살려 하이퍼파라미터 조정. 제출4에서 최고 점수가 기록됨
 - 편향된 데이터에서는 bootstrap을 하지 않으면 성능이 향상될 것으로 생각해 제출 5에서는 bootstrap을 false로 변경했으나 성능 차이 없음